<a href="https://colab.research.google.com/github/sanazgit/FER/blob/main/ResNet_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
import torch.nn as nn

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, rect=None, rect_local=None, num_classes=7, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):

        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])                         # 56x56x64
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])       # 28x28x128

        self.rect = rect
        self.layer3_1_1 = self._make_layer(block, 256, layers[2], stride=2,          # 14x14x256
                                           dilate=replace_stride_with_dilation[1])
        self.layer3_2_1 = self._make_layer(block, 512, layers[3], stride=1,          # 14x14x512
                                           dilate=replace_stride_with_dilation[1])

        self.layer3_1_2 = self._make_layer(block, 256, layers[2], stride=2,          # 14x14x256
                                           dilate=replace_stride_with_dilation[1])
        self.layer3_2_2 = self._make_layer(block, 512, layers[3], stride=1,          # 14x14x512
                                           dilate=replace_stride_with_dilation[1])

        self.layer3_1_3 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilate=replace_stride_with_dilation[1])
        self.layer3_2_3 = self._make_layer(block, 512, layers[3], stride=1,
                                           dilate=replace_stride_with_dilation[1])

        self.layer3_1_4 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilate=replace_stride_with_dilation[1])
        self.layer3_2_4 = self._make_layer(block, 512, layers[3], stride=1,
                                           dilate=replace_stride_with_dilation[1])

        self.rect_local = rect_local
        self.layer4_1 = self._make_layer(block, 256, layers[2], stride=2,          # 14x14x256
                                           dilate=replace_stride_with_dilation[1])
        self.layer4_2 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilate=replace_stride_with_dilation[1])
        self.layer4_3 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilate=replace_stride_with_dilation[1])
        self.layer4_4 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilate=replace_stride_with_dilation[1])
        self.layer4_5 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilate=replace_stride_with_dilation[1])





        self.layer5 = self._make_layer(block, 512, layers[3], stride=2,              # 7x7x512
                                       dilate=replace_stride_with_dilation[2])

        # Adjustment layers
        self.lp_adjust = nn.AdaptiveAvgPool2d((7, 7))  # adjust spatial dimensions for LP module
        self.ap_adjust = nn.Conv2d(256, 512, kernel_size=1)  # adjust channel dimensions for AP module





        # 3 branches
        self.conv2_1 = nn.Conv2d(512 * block.expansion, 64, kernel_size=1)
        self.conv2_2 = nn.Conv2d(512 * block.expansion, 64, kernel_size=1)
        self.conv2_3 = nn.Conv2d(512 * block.expansion, 64, kernel_size=1)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(64*3, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)


    def set_rect(self, rect):
      self.rect = rect

    def get_rect(self):
      return self.rect

    def set_rect_local(self, rect_local):
      self.rect_local = rect_local

    def get_rect_local(self):
      return self.rect_local

    def _forward_impl(self, x):

        #... Initial Convolution

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # 56x56x64


        #... Residual Blocks

        x = self.layer1(x)  # 56x56x64
        x = self.layer2(x)  # 28x28x128

      #... LP Module

        top_left = torch.zeros(x.shape[0], x.shape[1], 14, 14)
        top_right = torch.zeros(x.shape[0], x.shape[1], 14, 14)
        bottom_left = torch.zeros(x.shape[0], x.shape[1], 14, 14)
        bottom_right = torch.zeros(x.shape[0], x.shape[1], 14, 14)

        for i in range(len(x)):
          if self.rect[i][2] - self.rect[i][0] == 14 :
            top_left[i] = x[i][:, self.rect[i][1]:self.rect[i][3], self.rect[i][0]:self.rect[i][2]].clone()
          else:
            padding1 = 14 - (self.rect[i][2] - self.rect[i][0])
            pad_11 = nn.ZeroPad2d(padding=(padding1, 0, padding1, 0))
            top_left_min = x[i][:, self.rect[i][1]:self.rect[i][3], self.rect[i][0]:self.rect[i][2]].clone()
            top_left[i] = pad_11(top_left_min)

          if self.rect[i][4] - self.rect[i][6] == 14:
            top_right[i] = x[i][:, self.rect[i][5]:self.rect[i][7], self.rect[i][6]:self.rect[i][4]].clone()
          else:
            padding2 = 14 - (self.rect[i][4] - self.rect[i][6])
            pad_12 = nn.ZeroPad2d(padding=(0, padding2, padding2, 0))
            top_right_min = x[i][:, self.rect[i][5]:self.rect[i][7], self.rect[i][6]:self.rect[i][4]].clone()
            top_right[i] = pad_12(top_right_min)

          if self.rect[i][10] - self.rect[i][8] == 14:
            bottom_left[i] = x[i][:, self.rect[i][11]:self.rect[i][9], self.rect[i][8]:self.rect[i][10]].clone()
          else:
            padding1 = 14 - (self.rect[i][10] - self.rect[i][8])
            pad_21 = nn.ZeroPad2d(padding=(padding1, 0, 0, padding1))
            bottom_left_min = x[i][:, self.rect[i][11]:self.rect[i][9], self.rect[i][8]:self.rect[i][10]].clone()

            bottom_left[i] = pad_21(bottom_left_min)

          if self.rect[i][12] - self.rect[i][14] == 14:
            bottom_right[i] = x[i][:, self.rect[i][15]:self.rect[i][13], self.rect[i][14]:self.rect[i][12]].clone()
          else:
            padding2 = 14 - (self.rect[i][12] - self.rect[i][14])
            pad_22 = nn.ZeroPad2d(padding=(0, padding2, 0, padding2))
            bottom_right_min = x[i][:, self.rect[i][15]:self.rect[i][13], self.rect[i][14]:self.rect[i][12]].clone()
            bottom_right[i] = pad_22(bottom_right_min)


        top_left_out = self.layer3_1_1(top_left)
        top_left_out = self.layer3_2_1(top_left_out)

        top_right_out = self.layer3_1_2(top_right)
        top_right_out = self.layer3_2_2(top_right_out)

        bottom_left_out = self.layer3_1_3(bottom_left)
        bottom_left_out = self.layer3_2_3(bottom_left_out)

        bottom_right_out = self.layer3_1_4(bottom_right)
        bottom_right_out = self.layer3_2_4(bottom_right_out)

        # LP Adjustments:
        top_left_out = self.lp_adjust(top_left_out)
        top_right_out = self.lp_adjust(top_right_out)
        bottom_left_out = self.lp_adjust(bottom_left_out)
        bottom_right_out = self.lp_adjust(bottom_right_out)

        # Averaging outputs for LP
        lp_out = (top_left_out + top_right_out + bottom_left_out + bottom_right_out) / 4.0
        # OR
        # lp_out = torch.max(torch.stack([top_left_out, top_right_out, bottom_left_out, bottom_right_out]), dim=0)[0]

      #... AP Module

        rang = self.rect_local[0][1] - self.rect_local[0][0]
        eye1 = x[:, :, 0:rang, 0:rang].clone()
        eye2 = x[:, :, 0:rang, 0:rang].clone()
        eye_midd = x[:, :, 0:rang, 0:rang].clone()
        mouth1 = x[:, :, 0:rang, 0:rang].clone()
        mouth2 = x[:, :, 0:rang, 0:rang].clone()

        for i in range(x.shape[0]):
          eye1[i] = x[i][:, self.rect_local[i][2]:self.rect_local[i][3], self.rect_local[i][0]:self.rect_local[i][1]]
          eye2[i] = x[i][:, self.rect_local[i][6]:self.rect_local[i][7], self.rect_local[i][4]:self.rect_local[i][5]]
          eye_midd[i] = x[i][:, self.rect_local[i][10]:self.rect_local[i][11],
                        self.rect_local[i][8]:self.rect_local[i][9]]
          mouth1[i] = x[i][:, self.rect_local[i][14]:self.rect_local[i][15],
                      self.rect_local[i][12]:self.rect_local[i][13]]
          mouth2[i] = x[i][:, self.rect_local[i][18]:self.rect_local[i][19],
                      self.rect_local[i][16]:self.rect_local[i][17]]

        # 128→256
        eye1_out = self.layer4_1(eye1)
        eye2_out = self.layer4_2(eye2)
        eye_midd_out = self.layer4_3(eye_midd)
        mouth_out1 = self.layer4_4(mouth1)
        mouth_out2 = self.layer4_5(mouth2)

        # AP Adjustments:
        eye1_out = self.ap_adjust(eye1_out)
        eye2_out = self.ap_adjust(eye2_out)
        eye_midd_out = self.ap_adjust(eye_midd_out)
        mouth_out1 = self.ap_adjust(mouth_out1)
        mouth_out2 = self.ap_adjust(mouth_out2)

        # Averaging outputs for AP
        ap_out = (eye1_out + eye2_out + eye_midd_out + mouth_out1 + mouth_out2) / 5.0
        # OR
        # ap_out = torch.max(torch.stack([eye1_out, eye2_out, eye_midd_out, mouth_out1, mouth_out2]), dim=0)[0]

       # ... GP

        x_global = self.layer3_1_1 (x)
        gp_out = self.layer5(x_global)


        # Fusion:
        x_fused = (lp_out + ap_out + gp_out) / 3.0


        x_1 = self.conv2_1(x_fused)
        x_2 = self.conv2_2(x_fused)
        x_3 = self.conv2_3(x_fused)


        # Apply avgpool to each tensor to reduce spatial dimensions to 1x1
        x_fc1 = self.avgpool(x_1)     # Resultant shape: [batch_size, 64, 1, 1]
        x_fc2 = self.avgpool(x_2)     # Resultant shape: [batch_size, 64, 1, 1]
        x_fc3 = self.avgpool(x_3)     # Resultant shape: [batch_size, 64, 1, 1]


        # Flatten each tensor to prepare for concatenation
        x_fc1 = torch.flatten(x_fc1, 1)  # Resultant shape: [batch_size, 64]
        x_fc2 = torch.flatten(x_fc2, 1)  # Resultant shape: [batch_size, 64]
        x_fc3 = torch.flatten(x_fc3, 1)  # Resultant shape: [batch_size, 64]


        # Concatenate along the channel dimension (since each tensor is now 2D, this is dimension 1)
        x_concat = torch.cat([x_fc1, x_fc2, x_fc3], dim=1)  # Resultant shape: [batch_size, 192]
        # Pass the concatenated tensor through the fully connected layer
        x_out = self.fc(x_concat)  # Resultant shape: [batch_size, num_classes]

        return x_1, x_2, x_3, x_fc1, x_fc2, x_fc3, x_out

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress, **kwargs)


def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained=False, progress=True, **kwargs):
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)